## Part-of-Speech (POS) Tagging
POS tagging is the process of labeling words with grammatical categories (noun, verb, etc.) based on context. It resolves ambiguity - e.g., "book" can be a noun ("read a book") or verb ("book tickets").

**Example:**

Sentence: "They fish in the river"

Tags: They/PRP | fish/VBP | in/IN | the/DT | river/NN

PRP => Personal pronoun

VBP => Verb, non-3rd person singular present

IN => Preposition or subordinating conjunction

DT => Determiner

NN => Noun, singular or mass



## Hidden Markov Models (HMMs)
### Key Components:
- **Hidden States**: POS tags (NN, VB, etc.)
- **Observations**: Words in sentences
- **Transition Probabilities**: P(tag_i | tag_{i-1})
- **Emission Probabilities**: P(word | tag)[2][5]

### vs Markov Models:
| Feature          | Markov Model          | HMM                   |
|------------------|-----------------------|-----------------------|
| State Visibility | Fully observable       | Hidden                |
| Observations     | Same as states        | Different from states |
| Complexity       | Simple transitions    | Transition + Emission |
| Use Case         | Weather prediction    | Speech recognition    |

**Example HMM for POS Tagging:**

States: {NN, VB, DT} POS Tags (On Transitions)

Observations: {"the", "cat", "sits"} Emissions

Transition: P(VB|NN) = 0.3, P(NN|DT) = 0.6

Emission: P("cat"|NN) = 0.7, P("the"|DT) = 0.9


## Viterbi Algorithm
### Why Needed:
Computes the **most likely tag sequence** efficiently (O(nk²) vs O(kⁿ) brute force). Essential for decoding HMM states in realistic NLP tasks.

Core Idea

Given:

1. Observations: Words in a sentence (e.g., ["The", "cat", "sits"])

2.  Hidden States: POS tags (e.g., DT, NN, VB)

3. Transition Probabilities: P(tagi∣tagi−1)

4. Emission Probabilities: P(word∣tag)

The algorithm computes the optimal tag sequence by combining these probabilities efficiently

Key Steps
1. **Initialization**

Create two matrices:

* Viterbi Table dp: Stores maximum probabilities
* Backpointers bp: Tracks optimal previous states

For the first word w1:
dp[s]=P_initial(s)×P_emit(w1∣s)


where s = all possible tags

Word: "The"

Tags: DT (0.8), NN (0.1)

dp[0][DT] = 0.8 * 0.9 = 0.72  

dp[0][NN] = 0.1 * 0.1 = 0.01

2. **Recursion**

For each subsequent word wt:

dp[t][s]=max_⁡r∈tags(dp[t−1][r]×P_trans(s∣r)×P_emit(wt∣s))

Update bp[t][s] with the best r

Example:

Word: "cat"

Previous best: DT (0.72)

dp[1][NN] = 0.72 * P(NN|DT) * P("cat"|NN)

           = 0.72 * 0.6 * 0.7 = 0.3024

3. **Termination**

Find the maximum probability in the last column:

Best_Path_End=arg⁡max⁡_s(dp[n−1][s])

4. **Backtracking**

best_path = [best_final_tag]

for t in reversed(range(1, n)):

    best_path.insert(0, bp[t][best_path[0]])



In [ ]:
!pip install nltk numpy


In [ ]:
# Download required resources
import nltk
nltk.download('treebank')
nltk.download('punkt')
nltk.download('universal_tagset')
nltk.download('punkt_tab')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
# Import Required Libraries
import nltk
from nltk.corpus import treebank
from nltk.tokenize import word_tokenize
from collections import defaultdict # It will helpful in counting
import numpy as np
from sklearn.model_selection import train_test_split


In [ ]:
# Load and prepare dataset
tagged_sentences = treebank.tagged_sents(tagset='universal')  # Using universal tags
train_sentences, test_sentences = train_test_split(tagged_sentences, test_size=0.1, random_state=30)
train_sentences, val_sentences = train_test_split(train_sentences, test_size=0.1, random_state=30)

print(f"Total length of the dataset: {len(tagged_sentences)}")
print("Random Sample: ")
print(train_sentences[12])
print(f"Training Sample: {len(train_sentences)}")
print(f"Validation Sample: {len(val_sentences)}")
print(f"Test sample: {len(test_sentences)}")

Total length of the dataset: 3914
Random Sample: 
[('We', 'PRON'), ('also', 'ADV'), ('know', 'VERB'), ('that', 'ADP'), ('the', 'DET'), ('tort', 'NOUN'), ('system', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('lousy', 'ADJ'), ('way', 'NOUN'), ('0', 'X'), ('*', 'X'), ('to', 'PRT'), ('compensate', 'VERB'), ('victims', 'NOUN'), ('*T*-1', 'X'), ('anyway', 'ADV'), (';', '.'), ('some', 'DET'), ('win', 'VERB'), ('the', 'DET'), ('legal', 'ADJ'), ('lottery', 'NOUN'), (',', '.'), ('others', 'NOUN'), ('get', 'VERB'), ('much', 'ADV'), ('less', 'ADJ'), ('and', 'CONJ'), ('contingency-fee', 'ADJ'), ('lawyers', 'NOUN'), ('take', 'VERB'), ('a', 'DET'), ('big', 'ADJ'), ('cut', 'NOUN'), ('either', 'DET'), ('way', 'NOUN'), ('.', '.')]
Training Sample: 3169
Validation Sample: 353
Test sample: 392


In [ ]:
def compute_probabilities(sentences, alpha=1, beta=1):
    # === Data Collection Phase ===
    # Identify all unique tags and words in the training data
    all_tags = set()
    all_words = set()
    for sentence in sentences:
        for word, tag in sentence:
            all_tags.add(tag)    # Collect POS tags
            all_words.add(word)  # Collect vocabulary
    all_tags = list(all_tags)    # Convert to list for ordered access

    # === Counting Initialization ===
    # Create data structures for storing counts
    transition_counts = defaultdict(lambda: defaultdict(int))  # tag_i-1 → tag_i counts
    emission_counts = defaultdict(lambda: defaultdict(int))    # tag → word counts
    tag_counts = defaultdict(int)                              # tag → total occurrences

    # === Counting with Smoothing ===
    for sentence in sentences:
        prev_tag = '<s>'  # Start with sentence beginning marker

        for word, tag in sentence:
            # Transition smoothing: Add alpha to all possible transitions
            for t in all_tags:
                transition_counts[prev_tag][t] += alpha
            # Actual observed transition count
            transition_counts[prev_tag][tag] += 1  # Overwrite smoothed value

            # Emission smoothing: Add beta to all possible words (implicit)
            emission_counts[tag][word] += beta
            tag_counts[tag] += 1  # Total count for this tag

            prev_tag = tag  # Move to next position

        # End-of-sentence transition
        transition_counts[prev_tag]['</s>'] += 1

    # === Transition Probability Calculation ===
    transition_prob = defaultdict(dict)
    for prev_tag in transition_counts:
        # Total transitions from prev_tag (with smoothing denominator)
        total = sum(transition_counts[prev_tag].values()) + alpha * len(all_tags)

        for curr_tag in transition_counts[prev_tag]:
            # Subtract alpha because we added it during smoothing initialization
            prob = (transition_counts[prev_tag][curr_tag] - alpha) / total
            transition_prob[prev_tag][curr_tag] = prob

    # === Emission Probability Calculation ===
    emission_prob = defaultdict(dict)
    for tag in emission_counts:
        # Total emissions for tag (with smoothing denominator)
        total = tag_counts[tag] + beta * len(all_words)

        for word in emission_counts[tag]:
            emission_prob[tag][word] = emission_counts[tag][word] / total

    return transition_prob, emission_prob, all_tags


In [ ]:
def viterbi(sentence, transition_prob, emission_prob, all_tags):
    """
    Finds the most probable POS tag sequence using dynamic programming.

    Args:
        sentence: List of words to tag
        transition_prob: Precomputed transition probabilities
        emission_prob: Precomputed emission probabilities
        all_tags: List of all possible POS tags

    Returns:
        List of predicted tags for the input sentence
    """

    # === Initialization ===
    T = len(sentence)  # Number of words in the sentence
    num_tags = len(all_tags)

    # DP table: V[t][i] = max probability of being in tag i at position t
    V = np.zeros((T, num_tags))

    # Backpointer table: tracks index of best previous tag
    backpointer = np.zeros((T, num_tags), dtype=int)

    # Initialize first column (t=0)
    for i, tag in enumerate(all_tags):
        # Probability = P(start -> tag) * P(word0 | tag)
        V[0][i] = transition_prob['<s>'].get(tag, 1e-6) * \
                 emission_prob[tag].get(sentence[0], 1e-6)

    # === Forward Pass ===
    # Fill DP table from left to right
    for t in range(1, T):  # For each word position
        for i, curr_tag in enumerate(all_tags):  # Current tag candidates

            max_prob = -1
            best_prev_idx = 0

            # Find best previous tag
            for j, prev_tag in enumerate(all_tags):
                # Transition prob: prev_tag -> curr_tag
                trans_p = transition_prob[prev_tag].get(curr_tag, 1e-6)

                # Emission prob: current word given curr_tag
                emit_p = emission_prob[curr_tag].get(sentence[t], 1e-6)

                # Current probability calculation
                current_prob = V[t-1][j] * trans_p * emit_p

                # Track maximum probability
                if current_prob > max_prob:
                    max_prob = current_prob
                    best_prev_idx = j

            # Store results
            V[t][i] = max_prob
            backpointer[t][i] = best_prev_idx

    # === Backtracking ===
    # Reconstruct optimal path from DP tables
    best_path = []

    # Start from last word's best tag
    best_last_idx = np.argmax(V[-1])
    best_path.append(all_tags[best_last_idx])

    # Trace back through backpointers
    for t in range(T-1, 0, -1):
        best_prev_idx = backpointer[t][best_last_idx]
        best_path.insert(0, all_tags[best_prev_idx])  # Add to front
        best_last_idx = best_prev_idx

    return best_path


In [ ]:
# Evaluation function
def evaluate(model, test_data):
    correct = 0
    total = 0
    for sentence in test_data:
        words = [word for word, tag in sentence]
        true_tags = [tag for word, tag in sentence]
        pred_tags = viterbi(words, *model)
        correct += sum(1 for t, p in zip(true_tags, pred_tags) if t == p)
        total += len(sentence)
    return correct / total



In [ ]:
# Test the model
print("Validation Accuracy:", evaluate((transition_prob, emission_prob, all_tags), val_sentences))
print("Test Accuracy:", evaluate((transition_prob, emission_prob, all_tags), test_sentences))



Validation Accuracy: 0.9238095238095239
Test Accuracy: 0.929010170174202


In [ ]:
# Inference example
def tag_sentence(sentence):
    tokens = word_tokenize(sentence)
    tags = viterbi(tokens, transition_prob, emission_prob, all_tags)
    return list(zip(tokens, tags))

sample_sentence = "Ram is a good boy"
print("\nExample Tagging:")
print(tag_sentence(sample_sentence))


Example Tagging:
[('Ram', 'PRON'), ('is', 'VERB'), ('a', 'DET'), ('good', 'ADJ'), ('boy', 'NOUN')]
